# Separated

In [26]:
my_phone = "" #The phone number you'd like to receive texts on
country = "China" #Country name as given by US State Dept
province = "Hong Kong" #Works for province/state in Australia, Canada and China and dependencies of the Netherlands, the UK, France and Denmark. Otherwise, leave as empty string
desired_metric = "deaths" #deaths, confirmed, or recovered

In [27]:
from twilio.rest import Client

last_metric = 0
today_metric = 1

if desired_metric == "confirmed":
    pretty_dm = "cases"
elif desired_metric == "deaths":
    pretty_dm = desired_metric
elif desired_metric == "recovered":
    pretty_dm = "recoveries"

if province == '':
    text_body = f"The number of confirmed {pretty_dm} in {country} has at least doubled from {last_metric} to {today_metric}."
else:
    text_body = f"The number of confirmed {pretty_dm} in {province}, {country} has at least doubled from {last_metric} to {today_metric}."
    
def send_text(ph1, text_body):
    client = Client()

    message = client.messages.create(
        to="whatsapp:" + ph1, 
        from_='whatsapp:+14155238886',
        body=text_body
    )
    
send_text(ph1 = my_phone, text_body = text_body)

In [28]:
import pandas as pd

data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_' + desired_metric + '_global.csv')
today_data = data.iloc[:,-1]
today_metric = today_data.loc[data['Country/Region'] == country].loc[data['Province/State'] == province]
today_metric = int(today_metric)

In [30]:
from pathlib import Path

try:
    with open("last_metric", "r") as myFile:
        last_metric = int(myFile.read())
        if ((today_metric >= 2*last_metric) & (today_metric > 0)):
            print("send text")
            with open("last_metric", "w") as myFile:
                myFile.write(str(today_metric))
except FileNotFoundError:
    with open("last_metric", "w") as myFile:
        myFile.write(str(today_metric))

send text


# Consolidated

In [34]:
from twilio.rest import Client
import pandas as pd
from pathlib import Path

my_phone = "" #The phone number you'd like to receive texts on
country = "China" #Country name as given by US State Dept
province = "Hong Kong" #Works for province/state in Australia, Canada and China and dependencies of the Netherlands, the UK, France and Denmark. Otherwise, leave as empty string
desired_metric = "deaths" #deaths, confirmed, or recovered

if desired_metric == "confirmed":
    pretty_dm = "cases"
elif desired_metric == "deaths":
    pretty_dm = desired_metric
elif desired_metric == "recovered":
    pretty_dm = "recoveries"

if province == '':
    text_body = f"The number of confirmed {pretty_dm} in {country} has at least doubled from {last_metric} to {today_metric}."
else:
    text_body = f"The number of confirmed {pretty_dm} in {province}, {country} has at least doubled from {last_metric} to {today_metric}."
    
def send_text(ph1, text_body):
    client = Client(account_sid, auth_token)

    message = client.messages.create(
        to="whatsapp:" + ph1, 
        from_='whatsapp:+14155238886',
        body=text_body
    )

data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_' + desired_metric + '_global.csv')
today_metric = int(data.loc[data['Country/Region'] == country].loc[data['Province/State'] == province].iloc[:,-1])

try:
    with open("last_metric", "r") as myFile:
        last_metric = int(myFile.read())
        if ((today_metric >= 2*last_metric) & (today_metric > 0)):
            send_text(ph1 = my_phone, text_body = text_body)
            with open("last_metric", "w") as myFile:
                myFile.write(str(today_metric))
except FileNotFoundError:
    with open("last_metric", "w") as myFile:
        myFile.write(str(today_metric))